# 6-3,使用GPU训练模型

深度学习的训练过程常常非常耗时，一个模型训练几个小时是家常便饭，训练几天也是常有的事情，有时候甚至要训练几十天。

训练过程的耗时主要来自于两个部分，一部分来自数据准备，另一部分来自参数迭代。

当数据准备过程还是模型训练时间的主要瓶颈时，我们可以使用更多进程来准备数据。

当参数迭代过程成为训练时间的主要瓶颈时，我们通常的方法是应用GPU来进行加速。

In [ ]:
!pip install -q torchkeras
!pip install -q  -U torchmetrics

In [ ]:
import torch
import torchkeras
import torchmetrics

print("torch.__version__ = ", torch.__version__)
print("torchkeras.__version__ = ", torchkeras.__version__)
print("torchmetrics.__version__ = ", torchmetrics.__version__)

注：本节代码只能在有GPU的机器环境上才能正确执行。

对于没有GPU的同学，推荐使用

在Colab笔记本中：修改->笔记本设置->硬件加速器 中选择 GPU

可点击如下链接，直接在kaggle中运行范例代码。

https://www.kaggle.com/lyhue1991/pytorch-gpu-examples

Pytorch中使用GPU加速模型非常简单，只要将模型和数据移动到GPU上。核心代码只有以下几行。

```python
# 定义模型
... 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) # 移动模型到cuda

# 训练模型
...

features = features.to(device) # 移动数据到cuda
labels = labels.to(device) # 或者  labels = labels.cuda() if torch.cuda.is_available() else labels
...
```

如果要使用多个GPU训练模型，也非常简单。只需要在将模型设置为数据并行风格模型。
则模型移动到GPU上之后，会在每一个GPU上拷贝一个副本，并把数据平分到各个GPU上进行训练。核心代码如下。

```python
# 定义模型
... 

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model) # 包装为并行风格模型

# 训练模型
...
features = features.to(device) # 移动数据到cuda
labels = labels.to(device) # 或者 labels = labels.cuda() if torch.cuda.is_available() else labels
...
```

## 〇，GPU相关操作汇总

In [ ]:
import torch
from torch import nn

# 1，查看gpu信息
if_cuda = torch.cuda.is_available()
print("if_cuda=", if_cuda)

gpu_count = torch.cuda.device_count()
print("gpu_count=", gpu_count)


In [ ]:
# 创建一个随机初始化的100x100张量（tensor）
tensor = torch.rand((100, 100))

# 将张量移到GPU上
tensor_gpu = tensor.to("cuda:0")  # 或者使用 tensor_gpu = tensor.cuda()
# - to方法可以将张量移到指定的设备（这里是GPU的第一个设备cuda:0）
# - 使用.to("cuda:0") 或 .cuda() 方法是将张量移动到GPU的两种常见方式之一
# - tensor_gpu 保存了移动到GPU上的张量

# 打印张量所在设备信息
print(tensor_gpu.device)
# - 打印 tensor_gpu 的设备信息，应该显示为 cuda:0，表示在GPU上

# 检查张量是否在GPU上
print(tensor_gpu.is_cuda)
# - 打印 tensor_gpu 是否位于GPU上的布尔值，应该为True，表示张量在GPU上

# 将张量从GPU移到CPU上
tensor_cpu = tensor_gpu.to("cpu")  # 或者使用 tensor_cpu = tensor_gpu.cpu()
# - to方法可以将张量从一个设备移动到另一个设备，这里是从GPU移动到CPU
# - 使用.to("cpu") 或 .cpu() 方法是将张量从GPU移动到CPU的两种常见方式之一
# - tensor_cpu 保存了从GPU移到CPU上的张量

# 打印张量所在设备信息
print(tensor_cpu.device)
# - 打印 tensor_cpu 的设备信息，应该显示为 cpu，表示张量在CPU上



In [ ]:
# 创建一个简单的神经网络模型，这里是一个线性层，输入维度为2，输出维度为1
net = nn.Linear(2, 1)

# 打印模型的第一个参数张量是否位于GPU上
print(next(net.parameters()).is_cuda)
# - next(net.parameters()) 获取模型的第一个参数张量
# - .is_cuda 属性用于检查张量是否位于GPU上
# - 输出应该为False，因为模型还没有移动到GPU上

# 将模型中的全部参数张量移动到GPU上
net.to("cuda:0")
# - 使用 .to() 方法将模型中的全部参数张量依次移动到GPU上，这会改变模型本身的设备
# - 注意，不需要重新赋值为 net = net.to("cuda:0")，模型本身已经在GPU上了

# 打印模型的第一个参数张量是否位于GPU上
print(next(net.parameters()).is_cuda)
# - 打印模型的第一个参数张量是否位于GPU上的布尔值，应该为True，表示模型参数已经在GPU上

# 打印模型的第一个参数张量所在设备信息
print(next(net.parameters()).device)
# - 打印模型的第一个参数张量所在设备信息，应该显示为 cuda:0，表示参数在GPU上


In [ ]:
# 引入PyTorch库中的神经网络模块
import torch.nn as nn

# 创建一个简单的线性模型，输入维度为2，输出维度为1
linear = nn.Linear(2, 1)

# 打印模型的第一个参数张量所在设备信息
print(next(linear.parameters()).device)
# - next(linear.parameters()) 获取模型的第一个参数张量
# - .device 属性用于检查张量所在的设备，通常在CPU上

# 使用 nn.DataParallel 将模型支持多个GPU数据并行
model = nn.DataParallel(linear)
# - nn.DataParallel 可以将模型包装起来，以支持多个GPU数据并行
# - 这意味着可以在多个GPU上同时处理不同的数据批次，以加速训练

# 打印模型的设备IDs，即支持数据并行的GPU设备列表
print(model.device_ids)
# - 打印模型的设备IDs，应该显示为 [0]，表示模型在GPU 0 上进行数据并行
# - 通常，这个列表会包含多个GPU的设备ID，以便在多个GPU上并行运算

# 打印模型的第一个参数张量所在设备信息
print(next(model.module.parameters()).device)
# - next(model.module.parameters()) 获取模型中的第一个参数张量
# - .module 用于访问 nn.DataParallel 中包装的模型
# - .parameters() 用于获取模型的参数张量
# - 打印参数张量所在设备信息，应该显示为 cuda:0，表示参数在GPU 0 上

# 注意：当保存模型参数时，需要指定保存 model.module 的参数
torch.save(model.module.state_dict(), "model_parameter.pt")

# 创建一个新的线性模型
linear = nn.Linear(2, 1)

# 加载保存的模型参数
linear.load_state_dict(torch.load("model_parameter.pt"))
# - 使用 .load_state_dict() 方法加载保存的模型参数
# - 这样可以在新的模型上使用之前训练好的参数


## 一，矩阵乘法范例

下面分别使用CPU和GPU作一个矩阵乘法，并比较其计算效率。

In [ ]:
import time
import torch
from torch import nn

In [ ]:
# 创建两个随机初始化的矩阵 a 和 b
a = torch.rand((10000, 200))
b = torch.rand((200, 10000))

# 记录开始时间
tic = time.time()

# 执行矩阵乘法运算
c = torch.matmul(a, b)

# 记录结束时间
toc = time.time()

# 打印矩阵乘法运算所花费的时间
print(toc - tic)

# 打印矩阵 a 和 b 的设备信息
print(a.device)
print(b.device)


In [ ]:
# 引入PyTorch库
import torch
import time

# 检查GPU是否可用，如果可用，将使用GPU，否则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 在指定设备上创建随机矩阵 a
a = torch.rand((10000, 200), device=device)

# 在CPU上创建随机矩阵 b，然后将其移动到与矩阵 a 相同的设备上
b = torch.rand((200, 10000))
b = b.to(device)  # 或者使用 b = b.cuda() if torch.cuda.is_available() else b

# 记录开始时间
tic = time.time()

# 执行矩阵乘法运算
c = torch.matmul(a, b)

# 记录结束时间
toc = time.time()

# 打印矩阵乘法运算所花费的时间
print(toc - tic)

# 打印矩阵 a 和 b 的设备信息
print(a.device)
print(b.device)


## 二，线性回归范例

下面对比使用CPU和GPU训练一个线性回归模型的效率

### 1，使用CPU

In [ ]:
# 导入PyTorch库
import torch

# 准备数据
n = 1000000  # 样本数量

# 生成一个形状为 (n, 2) 的随机张量 X，其值在 [-5.0, 5.0) 之间，服从均匀分布
X = 10 * torch.rand([n, 2]) - 5.0

# 创建一个权重张量 w0，形状为 (1, 2)，用于线性组合
w0 = torch.tensor([[2.0, -3.0]])

# 创建一个偏置张量 b0，形状为 (1, 1)，用于线性组合
b0 = torch.tensor([[10.0]])

# 生成一个形状为 (n, 1) 的目标值张量 Y
# 通过将 X 与 w0 的转置相乘，再加上 b0，并添加正态分布噪声，来生成目标值
# @ 表示矩阵乘法，将 X 与 w0 的转置相乘，得到形状为 (n, 1) 的结果
# 然后，加上偏置 b0，最后添加一个服从均值为 0，标准差为 2.0 的正态分布噪声
Y = X @ w0.t() + b0 + torch.normal(0.0, 2.0, size=[n, 1])


In [ ]:
# 导入PyTorch库中的 nn 模块
import torch.nn as nn


# 定义线性回归模型类
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()

        # 创建模型参数 w 和 b，这些参数将在训练中学习
        # w 初始值是从与 w0 形状相同的正态分布中随机抽取的
        self.w = nn.Parameter(torch.randn_like(w0))

        # b 初始值是与 b0 形状相同的全零张量
        self.b = nn.Parameter(torch.zeros_like(b0))

    # 正向传播方法，定义了如何计算模型的输出
    def forward(self, x):
        # 计算预测值，x 与参数 w 的转置相乘再加上参数 b
        return x @ self.w.t() + self.b


# 创建一个 LinearRegression 类的实例化对象 linear，该对象代表一个线性回归模型
linear = LinearRegression()


In [ ]:
# 导入必要的库
import torch.optim as optim  # 导入优化器
import time  # 导入时间模块

# 定义优化器，使用 Adam 优化器来更新模型参数
# 参数 linear.parameters() 包含模型的权重和偏置参数
# lr 表示学习率，控制每次参数更新的步长
optimizer = optim.Adam(linear.parameters(), lr=0.1)

# 定义损失函数，使用均方误差损失函数 (Mean Squared Error)
loss_fn = nn.MSELoss()


# 定义训练函数
def train(epochs):
    tic = time.time()  # 记录训练开始时间
    for epoch in range(epochs):
        optimizer.zero_grad()  # 清零梯度，避免梯度累积

        # 计算模型对训练数据 X 的预测值
        Y_pred = linear(X)

        # 计算预测值与真实值 Y 之间的均方误差损失
        loss = loss_fn(Y_pred, Y)

        # 反向传播，计算梯度
        loss.backward()

        # 使用优化器来更新模型参数，实际执行梯度下降步骤
        optimizer.step()

        # 每训练 50 个 epoch 打印一次损失值
        if epoch % 50 == 0:
            print({"epoch": epoch, "loss": loss.item()})

    toc = time.time()  # 记录训练结束时间
    print("time used:", toc - tic)  # 打印训练用时


# 调用 train 函数，训练模型 500 个 epoch
train(500)


### 2，使用GPU

In [ ]:
# 导入PyTorch库
import torch

# 准备数据
n = 1000000  # 样本数量

# 生成一个形状为 (n, 2) 的张量 X，其中每个元素都在 -5.0 到 5.0 之间均匀分布
X = 10 * torch.rand([n, 2]) - 5.0

# 创建一个权重张量 w0，形状为 (1, 2)，用于线性变换
w0 = torch.tensor([[2.0, -3.0]])

# 创建一个偏置项张量 b0，形状为 (1, 1)
b0 = torch.tensor([[10.0]])

# 生成目标标签 Y，通过执行线性变换 X @ w0.t() + b0，并添加正态分布的噪声
# X @ w0.t() 表示 X 与 w0 的矩阵乘法，得到一个形状为 (n, 1) 的结果
# 然后，b0 被加到每个样本上
# 最后，torch.normal(0.0, 2.0, size=[n, 1]) 生成均值为 0.0，标准差为 2.0 的正态分布噪声
Y = X @ w0.t() + b0 + torch.normal(0.0, 2.0, size=[n, 1])

# 数据移动到GPU上
# 首先，检查是否可用GPU
print("torch.cuda.is_available() = ", torch.cuda.is_available())

# 如果GPU可用，将张量 X 和 Y 移动到GPU上进行加速计算
if torch.cuda.is_available():
    X = X.cuda()
    Y = Y.cuda()

# 打印设备信息，以确认数据是否成功移动到GPU
print("X.device:", X.device)  # 打印 X 张量所在的设备
print("Y.device:", Y.device)  # 打印 Y 张量所在的设备


In [ ]:
# 导入PyTorch库中的 nn 模块
import torch.nn as nn


# 定义线性回归模型
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()

        # 定义模型的权重参数 w，使用 nn.Parameter 包装张量，初始化为与 w0 形状相同的随机张量
        self.w = nn.Parameter(torch.randn_like(w0))

        # 定义模型的偏置参数 b，使用 nn.Parameter 包装张量，初始化为与 b0 形状相同的全零张量
        self.b = nn.Parameter(torch.zeros_like(b0))

    # 定义正向传播方法，计算模型的输出
    def forward(self, x):
        return x @ self.w.t() + self.b


# 创建线性回归模型实例
linear = LinearRegression()

# 移动模型到GPU上（如果GPU可用的话）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 使用 .to() 方法将模型移动到指定的设备（GPU或CPU）
linear.to(device)

# 查看模型是否已经移动到GPU上，通过检查模型参数中的第一个参数的 .is_cuda 属性
# 如果第一个参数位于GPU上，那么整个模型通常也会在GPU上执行
print("if on cuda:", next(linear.parameters()).is_cuda)


In [ ]:
# 导入所需的时间模块
import time

# 定义优化器，使用 Adam 优化器来更新模型参数
# 参数 linear.parameters() 返回模型的可学习参数（包括 w 和 b）
# lr=0.1 是学习率的设置
optimizer = torch.optim.Adam(linear.parameters(), lr=0.1)

# 定义损失函数，使用均方误差损失 (MSE Loss)
loss_fn = nn.MSELoss()


# 定义训练函数，接受一个参数 epoches，表示训练的总轮数
def train(epoches):
    tic = time.time()
    for epoch in range(epoches):
        # 每轮训练前先将梯度清零，以免梯度累积
        optimizer.zero_grad()

        # 通过模型进行正向传播，计算预测值 Y_pred
        Y_pred = linear(X)

        # 计算预测值 Y_pred 与真实标签 Y 之间的均方误差损失
        loss = loss_fn(Y_pred, Y)

        # 反向传播，计算梯度
        loss.backward()

        # 使用优化器更新模型参数
        optimizer.step()

        # 每隔 50 轮打印一次当前轮数和损失
        if epoch % 50 == 0:
            print({"epoch": epoch, "loss": loss.item()})

    toc = time.time()
    print("time used:", toc - tic)


# 调用训练函数进行模型训练，总共训练 500 轮
train(500)


## 三，图片分类范例

In [ ]:
# 导入PyTorch库
import torch

# 导入PyTorch的图像处理工具包
import torchvision

# 从torchvision模块导入transforms，transforms包含了图像预处理和数据增强的功能
from torchvision import transforms


In [ ]:
# 导入PyTorch库
import torch

# 导入PyTorch的计算机视觉工具包
import torchvision
from torchvision import transforms

# 定义数据预处理操作，这里使用transforms.Compose()将多个预处理操作组合在一起
transform = transforms.Compose([transforms.ToTensor()])

# 创建训练集数据集对象
# - root: 数据集存储的根目录
# - train=True: 表示加载训练集数据
# - download=True: 如果数据不存在，会自动下载
# - transform=transform: 使用之前定义的数据预处理操作
ds_train = torchvision.datasets.MNIST(root="mnist/", train=True, download=True, transform=transform)

# 创建验证集数据集对象
# - root: 数据集存储的根目录
# - train=False: 表示加载验证集数据
# - download=True: 如果数据不存在，会自动下载
# - transform=transform: 使用之前定义的数据预处理操作
ds_val = torchvision.datasets.MNIST(root="mnist/", train=False, download=True, transform=transform)

# 创建训练集数据加载器
# - ds_train: 训练集数据集对象
# - batch_size=128: 每个批次中包含128个样本
# - shuffle=True: 在每个时代(epoch)开始时打乱数据
# - num_workers=2: 使用2个工作进程来加载数据，加速数据加载
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=128, shuffle=True, num_workers=2)

# 创建验证集数据加载器
# - ds_val: 验证集数据集对象
# - batch_size=128: 每个批次中包含128个样本
# - shuffle=False: 不需要在验证集上打乱数据
# - num_workers=2: 使用2个工作进程来加载数据，加速数据加载
dl_val = torch.utils.data.DataLoader(ds_val, batch_size=128, shuffle=False, num_workers=2)

# 打印训练集和验证集的样本数量
print(len(ds_train))
print(len(ds_val))


In [ ]:
# 定义一个函数用于创建神经网络模型
def create_net():
    # 创建一个神经网络模型，使用Sequential容器来构建一个层序列
    net = nn.Sequential()

    # 添加卷积层1，输入通道数为1（MNIST图像是单通道的灰度图像），输出通道数为32，卷积核大小为3x3
    net.add_module("conv1", nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3))

    # 添加最大池化层1，池化核大小为2x2，步幅为2
    net.add_module("pool1", nn.MaxPool2d(kernel_size=2, stride=2))

    # 添加卷积层2，输入通道数为32，输出通道数为64，卷积核大小为5x5
    net.add_module("conv2", nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5))

    # 添加最大池化层2，池化核大小为2x2，步幅为2
    net.add_module("pool2", nn.MaxPool2d(kernel_size=2, stride=2))

    # 添加2D Dropout层，以防止过拟合，丢弃率为0.1
    net.add_module("dropout", nn.Dropout2d(p=0.1))

    # 添加自适应最大池化层，将特征图大小调整为(1, 1)
    net.add_module("adaptive_pool", nn.AdaptiveMaxPool2d((1, 1)))

    # 添加Flatten层，将特征图展平为一维向量
    net.add_module("flatten", nn.Flatten())

    # 添加全连接层1，输入特征数为64，输出特征数为32
    net.add_module("linear1", nn.Linear(64, 32))

    # 添加ReLU激活函数
    net.add_module("relu", nn.ReLU())

    # 添加全连接层2，输入特征数为32，输出特征数为10（对应于10个类别的输出）
    net.add_module("linear2", nn.Linear(32, 10))

    return net


# 调用create_net函数创建神经网络模型
net = create_net()

# 打印神经网络模型的结构
print(net)


### 1，使用CPU进行训练

In [ ]:
# 导入所需的库和模块
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from copy import deepcopy
from torchmetrics import Accuracy


# 定义一个用于打印日志的函数
def printlog(info):
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n" + "==========" * 8 + "%s" % nowtime)
    print(str(info) + "\n")


# 创建神经网络模型（这里假设create_net()函数已经在前面定义过）
net = create_net()

# 定义损失函数（交叉熵损失）
loss_fn = nn.CrossEntropyLoss()

# 定义优化器（Adam优化器）
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# 定义用于评估模型性能的指标（准确率）
metrics_dict = {"acc": Accuracy(task='multiclass', num_classes=10)}

# 训练轮数
epochs = 3

# 检查点保存路径
ckpt_path = 'checkpoint.pt'

# 提前停止（early stopping）相关设置
monitor = "val_acc"
patience = 1
mode = "max"

# 用于保存训练历史的字典
history = {}

# 开始训练循环
for epoch in range(1, epochs + 1):
    printlog("Epoch {0} / {1}".format(epoch, epochs))

    # 1. 训练
    net.train()

    total_loss, step = 0, 0

    # 使用tqdm创建一个进度条以显示训练进度
    loop = tqdm(enumerate(dl_train), total=len(dl_train), file=sys.stdout)
    train_metrics_dict = deepcopy(metrics_dict)

    for i, batch in loop:
        features, labels = batch

        # 前向传播
        preds = net(features)
        loss = loss_fn(preds, labels)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 计算指标（例如，准确率）
        step_metrics = {"train_" + name: metric_fn(preds, labels).item()
                        for name, metric_fn in train_metrics_dict.items()}

        step_log = dict({"train_loss": loss.item()}, **step_metrics)

        total_loss += loss.item()

        step += 1
        if i != len(dl_train) - 1:
            loop.set_postfix(**step_log)
        else:
            epoch_loss = total_loss / step
            epoch_metrics = {"train_" + name: metric_fn.compute().item()
                             for name, metric_fn in train_metrics_dict.items()}
            epoch_log = dict({"train_loss": epoch_loss}, **epoch_metrics)
            loop.set_postfix(**epoch_log)

            for name, metric_fn in train_metrics_dict.items():
                metric_fn.reset()

    for name, metric in epoch_log.items():
        history[name] = history.get(name, []) + [metric]

    # 2. 验证
    net.eval()

    total_loss, step = 0, 0
    loop = tqdm(enumerate(dl_val), total=len(dl_val), file=sys.stdout)
    val_metrics_dict = deepcopy(metrics_dict)

    with torch.no_grad():
        for i, batch in loop:
            features, labels = batch

            # 前向传播
            preds = net(features)
            loss = loss_fn(preds, labels)

            # 计算指标
            step_metrics = {"val_" + name: metric_fn(preds, labels).item()
                            for name, metric_fn in val_metrics_dict.items()}

            step_log = dict({"val_loss": loss.item()}, **step_metrics)

            total_loss += loss.item()
            step += 1
            if i != len(dl_val) - 1:
                loop.set_postfix(**step_log)
            else:
                epoch_loss = (total_loss / step)
                epoch_metrics = {"val_" + name: metric_fn.compute().item()
                                 for name, metric_fn in val_metrics_dict.items()}
                epoch_log = dict({"val_loss": epoch_loss}, **epoch_metrics)
                loop.set_postfix(**epoch_log)

                for name, metric_fn in val_metrics_dict.items():
                    metric_fn.reset()

    epoch_log["epoch"] = epoch
    for name, metric in epoch_log.items():
        history[name] = history.get(name, []) + [metric]

    # 3. 提前停止
    arr_scores = history[monitor]
    best_score_idx = np.argmax(arr_scores) if mode == "max" else np.argmin(arr_scores)
    if best_score_idx == len(arr_scores) - 1:
        torch.save(net.state_dict(), ckpt_path)
        print("<<<<<< reach best {0} : {1} >>>>>>".format(monitor,
                                                          arr_scores[best_score_idx]))
    if len(arr_scores) - best_score_idx > patience:
        print("<<<<<< {} without improvement in {} epoch, early stopping >>>>>>".format(
            monitor, patience))
        break
    net.load_state_dict(torch.load(ckpt_path))

# 将训练历史保存为DataFrame
dfhistory = pd.DataFrame(history)


### 2，使用GPU进行训练

In [ ]:
# 导入所需的库和模块
import sys, time
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm

import torch
from torch import nn
from copy import deepcopy
from torchmetrics import Accuracy


# 定义一个用于打印日志的函数
def printlog(info):
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n" + "==========" * 8 + "%s" % nowtime)
    print(str(info) + "\n")


# 创建神经网络模型（假设create_net()函数已在之前定义过）
net = create_net()

# 定义损失函数（交叉熵损失）
loss_fn = nn.CrossEntropyLoss()

# 定义优化器（Adam优化器）
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# 定义用于评估模型性能的指标（准确率）
metrics_dict = {"acc": Accuracy(task='multiclass', num_classes=10)}

# 移动模型和相关组件到GPU（如果可用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
loss_fn.to(device)
for name, fn in metrics_dict.items():
    fn.to(device)

# 训练轮数
epochs = 5

# 检查点保存路径
ckpt_path = 'checkpoint.pt'

# 提前停止（early stopping）相关设置
monitor = "val_acc"
patience = 1
mode = "max"

# 用于保存训练历史的字典
history = {}

for epoch in range(1, epochs + 1):
    printlog("Epoch {0} / {1}".format(epoch, epochs))

    # 1. 训练
    net.train()

    total_loss, step = 0, 0

    # 使用tqdm创建一个进度条以显示训练进度
    loop = tqdm(enumerate(dl_train), total=len(dl_train), file=sys.stdout)
    train_metrics_dict = deepcopy(metrics_dict)

    for i, batch in loop:
        features, labels = batch

        # 移动数据到GPU上
        features = features.to(device)
        labels = labels.to(device)

        # 前向传播
        preds = net(features)
        loss = loss_fn(preds, labels)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 计算指标（例如，准确率）
        step_metrics = {"train_" + name: metric_fn(preds, labels).item()
                        for name, metric_fn in train_metrics_dict.items()}

        step_log = dict({"train_loss": loss.item()}, **step_metrics)

        total_loss += loss.item()

        step += 1
        if i != len(dl_train) - 1:
            loop.set_postfix(**step_log)
        else:
            epoch_loss = total_loss / step
            epoch_metrics = {"train_" + name: metric_fn.compute().item()
                             for name, metric_fn in train_metrics_dict.items()}
            epoch_log = dict({"train_loss": epoch_loss}, **epoch_metrics)
            loop.set_postfix(**epoch_log)

            for name, metric_fn in train_metrics_dict.items():
                metric_fn.reset()

    for name, metric in epoch_log.items():
        history[name] = history.get(name, []) + [metric]

    # 2. 验证
    net.eval()

    total_loss, step = 0, 0
    loop = tqdm(enumerate(dl_val), total=len(dl_val), file=sys.stdout)
    val_metrics_dict = deepcopy(metrics_dict)

    with torch.no_grad():
        for i, batch in loop:
            features, labels = batch

            # 移动数据到GPU上
            features = features.to(device)
            labels = labels.to(device)

            # 前向传播
            preds = net(features)
            loss = loss_fn(preds, labels)

            # 计算指标
            step_metrics = {"val_" + name: metric_fn(preds, labels).item()
                            for name, metric_fn in val_metrics_dict.items()}

            step_log = dict({"val_loss": loss.item()}, **step_metrics)

            total_loss += loss.item()
            step += 1
            if i != len(dl_val) - 1:
                loop.set_postfix(**step_log)
            else:
                epoch_loss = (total_loss / step)
                epoch_metrics = {"val_" + name: metric_fn.compute().item()
                                 for name, metric_fn in val_metrics_dict.items()}
                epoch_log = dict({"val_loss": epoch_loss}, **epoch_metrics)
                loop.set_postfix(**epoch_log)

                for name, metric_fn in val_metrics_dict.items():
                    metric_fn.reset()

    epoch_log["epoch"] = epoch
    for name, metric in epoch_log.items():
        history[name] = history.get(name, []) + [metric]

    # 3. 提前停止
    arr_scores = history[monitor]
    best_score_idx = np.argmax(arr_scores) if mode == "max" else np.argmin(arr_scores)
    if best_score_idx == len(arr_scores) - 1:
        torch.save(net.state_dict(), ckpt_path)
        print("<<<<<< reach best {0} : {1} >>>>>>".format(monitor,
                                                          arr_scores[best_score_idx]))
    if len(arr_scores) - best_score_idx > patience:
        print("<<<<<< {} without improvement in {} epoch, early stopping >>>>>>".format(
            monitor, patience))
        break
    net.load_state_dict(torch.load(ckpt_path))

# 将训练历史保存为DataFrame
dfhistory = pd.DataFrame(history)


## 四，torchkeras.KerasModel中使用GPU

从上面的例子可以看到，在pytorch中使用GPU并不复杂，但对于经常炼丹的同学来说，模型和数据老是移来移去还是蛮麻烦的。

一不小心就会忘了移动某些数据或者某些module，导致报错。

torchkeras.KerasModel 在设计的时候考虑到了这一点，如果环境当中存在可用的GPU，会自动使用GPU，反之则使用CPU。

通过引入accelerate的一些基础功能，torchkeras.KerasModel以非常优雅的方式在GPU和CPU之间切换。

详细实现可以参考torchkeras.KerasModel的源码。

In [ ]:
# 导入 accelerate 库
import accelerate

# 创建 Accelerator 实例，它将帮助自动选择合适的硬件加速器（例如 GPU）
accelerator = accelerate.Accelerator()

# 打印加速器设备信息
print(accelerator.device)


In [ ]:
# 从 torchkeras 库中导入 KerasModel 类
from torchkeras import KerasModel

# 导入需要的库和模块
from torchmetrics import Accuracy

# 创建神经网络模型
net = create_net()

# 使用 KerasModel 类创建模型，设置损失函数、指标（准确率）、优化器等参数
model = KerasModel(net,
                   loss_fn=nn.CrossEntropyLoss(),
                   metrics_dict={"acc": Accuracy(task='multiclass', num_classes=10)},
                   optimizer=torch.optim.Adam(net.parameters(), lr=0.01))

# 使用 fit() 方法来进行模型训练
model.fit(
    train_data=dl_train,  # 训练数据集
    val_data=dl_val,  # 验证数据集
    epochs=10,  # 训练轮数
    patience=3,  # 提前停止的耐心轮数
    monitor="val_acc",  # 监控的性能指标（这里是验证准确率）
    mode="max"  # 监控指标的模式（最大化验证准确率）
)
